In [2]:
# import pandas as pd
# from functions import load_data, clean_data, style_table

# def main():
#     # Load and clean data
#     url = "https://www.sharkattackfile.net/spreadsheets/GSAF5.xls"
#     df_sa = load_data(url)
#     df_sa = clean_data(df_sa)

#     # Ask the user if they want to display the table
#     show_table = input("Show table? Y/N: ").strip().lower()

#     if show_table == 'y':
#         styled_df = style_table(df_sa)
#         display(styled_df)
    
#     display(df_sa.head(10))

# if __name__ == "__main__":
#     main()

In [3]:
import pandas as pd

pd.set_option('display.max_rows', None)

url = "https://www.sharkattackfile.net/spreadsheets/GSAF5.xls"

df_sa = pd.read_excel(url)

df_sa.head()

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,15 Mar 2024,2024.0,Unprovoked,AUSTRALIA,Queensland,Bargara Beach,Swimming,Brooklyn Sauer,F,13,...,Tiger shark,"Yahoo News, 3/15/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,04 Mar 2024,2024.0,Unprovoked,USA,Hawaii,"Old Man's, Waikiki",Surfing,Matthew White,M,NaN,...,Tiger shark 8',"Surfer, 3/6/2024F",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,02 Mar-2024,2024.0,Unprovoked,USA,Hawaii,"Rainbows, Oahu",Swimming,NaN,F,11,...,3' to 4' shark,"Hawaii News Now, 3/4/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25 Feb-2024,2024.0,Unprovoked,AUSTRALIA,Western Australia,"Sandlnd Island, Jurian Bay",NaN,female,F,46,...,Tiger shark,"WA Today, 2/26/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14 Feb-2024,2024.0,Unprovoked,INDIA,Maharashtra,"Vaitarna River, Palghar District",Fishing,Vicky Suresh Govari,M,32,...,"Bull shark, 7'","Times of India, 2/14/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# COL NAMES

df_sa.rename(columns= lambda x : x.lower().replace(" ", "_"), inplace=True)

df_sa.columns

Index(['date', 'year', 'type', 'country', 'state', 'location', 'activity',
       'name', 'sex', 'age', 'injury', 'unnamed:_11', 'time', 'species_',
       'source', 'pdf', 'href_formula', 'href', 'case_number', 'case_number.1',
       'original_order', 'unnamed:_21', 'unnamed:_22'],
      dtype='object')

In [5]:
# YEAR

df_sa["year"] = df_sa["year"].fillna(0).astype(int)             # I fill NA with zeros in order to avoid errors when parsing float to int

df_sa["year"].value_counts()

year
0       156
2015    143
2017    139
2016    131
2011    128
2014    126
2018    124
2008    122
2013    122
2009    120
2012    117
2019    114
2007    113
2021    109
2023    108
2006    103
2005    103
2010    101
2020    101
2022     98
2000     97
1959     93
1960     93
2004     92
2003     92
2001     92
2002     88
1962     86
1961     78
1995     76
1999     66
1964     66
1998     65
1996     61
1963     61
1966     58
1997     57
1994     57
1992     56
1988     56
1993     56
1958     54
1989     53
1965     51
1956     51
1983     50
1975     49
1981     49
1967     48
1968     47
1954     45
1950     43
1955     43
1970     42
1982     42
1957     41
1942     41
1984     41
1974     39
1976     39
1986     39
1990     39
1991     39
1929     38
1985     37
1953     36
1980     36
1972     36
1987     35
1951     32
1935     32
1936     32
1944     31
1949     31
1969     30
1937     30
1947     30
1971     30
1952     29
1931     29
1948     29
1934     28
1943     28

In [6]:
df_sa["type"].value_counts()

type
Unprovoked             5107
Provoked                635
Invalid                 552
Watercraft              355
Sea Disaster            242
Questionable             22
Boat                      7
 Provoked                 2
?                         1
Unconfirmed               1
Unverified                1
Under investigation       1
Name: count, dtype: int64

In [7]:
# TYPE

df_sa["type"] = df_sa["type"].str.strip()

df_sa = df_sa.loc[df_sa["type"].isin(df_sa["type"].value_counts()[lambda x: x >= 30].index)]           # I´m deleting results whose nº of repetitions is above 30

df_sa["type"].value_counts()


type
Unprovoked      5107
Provoked         637
Invalid          552
Watercraft       355
Sea Disaster     242
Name: count, dtype: int64

In [8]:
# COUNTRY

def clean_str_punctuation(df):
    mytable = str.maketrans('', '', '¡¿.,!?;')

    df = df.str.strip().str.title().str.translate(mytable)

    return df


df_sa["country"] = clean_str_punctuation(df_sa["country"])


df_sa["country"].value_counts()




country
Usa                                     2520
Australia                               1472
South Africa                             596
New Zealand                              145
Papua New Guinea                         136
Bahamas                                  133
Brazil                                   122
Mexico                                   102
Italy                                     72
Fiji                                      69
New Caledonia                             65
Philippines                               64
Reunion                                   60
Egypt                                     49
Cuba                                      48
Spain                                     46
Mozambique                                46
India                                     41
French Polynesia                          37
Japan                                     35
Croatia                                   35
Panama                                    32
So

In [9]:
# STATE

df_sa["state"] = clean_str_punctuation(df_sa["state"])

df_sa["state"].value_counts()

state
Florida                                                           1170
New South Wales                                                    518
Queensland                                                         346
Hawaii                                                             339
California                                                         321
Western Australia                                                  229
Kwazulu-Natal                                                      218
Western Cape Province                                              196
South Carolina                                                     172
Eastern Cape Province                                              168
North Carolina                                                     120
South Australia                                                    115
Victoria                                                            96
Pernambuco                                                          80


In [10]:
# LOCATION

df_sa["location"] = clean_str_punctuation(df_sa["location"])

df_sa["location"].value_counts()

AttributeError: Can only use .str accessor with string values!